In [1]:
import torch
import gradio as gr
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
from PIL import Image, ImageEnhance
from itertools import permutations, combinations
import numpy as np
import uuid

In [2]:
model = torch.hub.load('ultralytics/yolov5', 'custom', path='yolov5s_nhat.pt', force_reload=True) 

Downloading: "https://github.com/ultralytics/yolov5/archive/master.zip" to /home/aia/.cache/torch/hub/master.zip
YOLOv5 🚀 2023-3-22 Python-3.8.16 torch-1.11.0+cu102 CPU

Fusing layers... 
Model summary: 157 layers, 7150369 parameters, 0 gradients, 16.2 GFLOPs
Adding AutoShape... 


In [3]:
path_52cards = "52cards/"
prefix = "800px-Playing_card_"
suffix = ".svg.png"

In [4]:
suits = {"spade":0, "club":1, "diamond":2, "heart":3}
numbs = {"2":0, "3":1, "4":2, "5":3, "6":4, "7":5, "8":6,
         "9":7, "10":8, "J":9, "Q":10, "K":11, "A":12}

ranks = {
    "0_highcard": 0, "0_pair": 1,
    "1_highcard": 2, "1_pair": 3,
    "2_highcard": 4, "2_pair": 5,
    "3_highcard": 6, "3_pair": 7,
    "4_highcard": 8, "4_pair": 9,
    "5_highcard": 10, "5_pair": 11,
    "6_highcard": 12, "6_pair": 13,
    "7_highcard": 14, "7_pair": 15,
    "8_highcard": 16, "8_pair": 17,
    "9_highcard": 18, "9_pair": 19,
    "three_of_images": 20,
    "straight": 21,
    "three_of_a_kind": 22,
}

all_of_straights = [("2", "3", "A")]
all_of_straights += [("2", "3", "4")]
all_of_straights += [("3", "4", "5")]
all_of_straights += [("4", "5", "6")]
all_of_straights += [("5", "6", "7")]
all_of_straights += [("6", "7", "8")]
all_of_straights += [("7", "8", "9")]
all_of_straights += [("8", "9", "10")]
all_of_straights += [("9", "10", "J")]
all_of_straights += [("10", "J", "Q")]
all_of_straights += [("J", "Q", "K")]
all_of_straights += [("Q", "K", "A")]

def intt(card):
    try:
        num = int(card)
    except:
        if card == "A":
            return 1
        else:
            return 0
    return num % 10


def card_split(suit_card):
    tmp = suit_card.split("_")
    return tmp[0], tmp[1]


def four_of_a_kind(hands):
    nums = [card.split("_")[1] for card in hands]
    numx = None
    for i, nu in enumerate(nums[:-3]):
        if nu == nums[i+3]:
            numx = nu
    return numx


def identify_combo(combo):
    card_1, card_2, card_3 = combo
    s1, n1 = card_split(card_1)
    s2, n2 = card_split(card_2)
    s3, n3 = card_split(card_3)
    
    if n1 == n3:
        return "three_of_a_kind", card_1
    
    if n1=="2" and n2 == "3" and n3 =="A":
        return "straight", card_2
    if (n1, n2, n3) in all_of_straights:
        return "straight", card_3
    
    # There's no such thing as "J", "Q" and "K" appearing together
    jqk = ["J", "Q", "K"]
    if n1 in jqk and n2 in jqk and n3 in jqk:
        if n1 == n2:
            return "three_of_images", card_2
        if n2 == n3:
            return "three_of_images", card_3
    
    cards_sum = (intt(n1) + intt(n2) + intt(n3)) % 10
    if n1 == n2:
        return str(cards_sum) + "_pair", card_2
    if n2 == n3:
        return str(cards_sum) + "_pair", card_3

    return str(cards_sum) + "_highcard", card_3


def scores_computation(combo):
    identify, card = identify_combo(combo)
    s, n = card_split(card)
    score = ranks[identify] + (numbs[n]*4+suits[s]) / 52
    return score


def vip_permutations(hands):
    
    # init
    A23_min_score = ranks["straight"]+(numbs["3"]*4+0)/52
    A23_max_score = ranks["straight"]+(numbs["3"]*4+3)/52
    
    top_4 = [None, None, None, None]
    top_4_scores = [0, 0, 0, 0]
    
    comboes_1 = list(combinations(hands, 3))
    for c0, c1, c2 in comboes_1:
        combo_1 = (c0, c1, c2)
        tmp_hands = hands.copy()
        tmp_hands.remove(c0)
        tmp_hands.remove(c1)
        tmp_hands.remove(c2)
        comboes_2 = list(combinations(tmp_hands, 3))
        for c3, c4, c5 in comboes_2:
            combo_2 = (c3, c4, c5)

            combo_3 = tmp_hands.copy()
            combo_3.remove(c3)
            combo_3.remove(c4)
            combo_3.remove(c5)
            combo_3 = tuple(combo_3)
            c6, c7, c8 = combo_3
            
            score_1 = scores_computation(combo_1)
            score_2 = scores_computation(combo_2)
            score_3 = scores_computation(combo_3)
            
            if score_1 < score_2 and score_2 < score_3:
                if A23_min_score <= score_1 and score_1 <= A23_max_score:
                    combo_1 = (c2, c0, c1)
                if A23_min_score <= score_2 and score_2 <= A23_max_score:
                    combo_2 = (c5, c3, c4)
                if A23_min_score <= score_3 and score_3 <= A23_max_score:
                    combo_3 = (c8, c6, c7)
                    
                tmp_h = [combo_1, combo_2, combo_3]
                tmp_s = score_1 + score_2 + score_3
                
                for k in range(4):
                    if top_4_scores[k] < tmp_s:
                        if k > 0:
                            top_4[k-1] = top_4[k]
                            top_4_scores[k-1] = top_4_scores[k]
                        top_4[k] = tmp_h
                        top_4_scores[k] = tmp_s
                        
    return top_4

In [5]:
test_path = "test/"
results_path = "results/"
dragon_path = path_52cards + "dragon.png"
khang_path = path_52cards + "KhangBao.jpg"
khang_img = mpimg.imread(khang_path)

import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec

def show_hand(hands, result_path):
    
    fig = plt.figure(figsize=(8, 40))
    outer = gridspec.GridSpec(4, 1, wspace=0.2, hspace=0.2)
    outer.update(left=0.1, right=0.9, top=0.985, bottom=0.015)
    
    size_ = min(4, len(hands))
    for i, hand in enumerate(hands[:size_]):
        inner = gridspec.GridSpecFromSubplotSpec(3, 3, subplot_spec=outer[3-i], wspace=0.1, hspace=0.05)
        for j in range(9):
            ax = plt.Subplot(fig, inner[j])
            ax.axis("off")
            
            card = hand[j//3][j%3]
            card_path = path_52cards + prefix + card + suffix
            img = mpimg.imread(card_path)
            ax.imshow(img)
            ax.set_xticks([])
            ax.set_yticks([])
            fig.add_subplot(ax)

    plt.axis("off")
    fig.savefig(result_path)
    im = Image.open(result_path)
    plt.show()
    
    return im


def sort_n2(a):
    
    def cards_comparison(card_1, card_2):
        s1, n1 = card_split(card_1)
        s2, n2 = card_split(card_2)
        if numbs[n1] > numbs[n2]:
            return 1
        elif numbs[n1] < numbs[n2]:
            return -1
        else:
            if suits[s1] > suits[s2]:
                return 1
            elif suits[s1] < suits[s2]:
                return -1
        return 0
    
    
    for i in range(0, len(a)-1):
        for j in range(i+1, len(a)):
            if cards_comparison(a[i], a[j]) > 0:
                a[i], a[j] = a[j], a[i]
    

def flip(img):
    try:
        uuid_generator = str(uuid.uuid4())
        image_path = test_path + uuid_generator + ".jpg"
        result_path = results_path + uuid_generator + ".jpg"

        width, height = img.size
        if width == 0 or height == 0:
            return khang_img
        if height > 640:
            width = width * 640 // height
            height = 640
            img = img.resize((width, height))

    #     enhancer = ImageEnhance.Brightness(img)
    #     factor = 1.0
    #     img = enhancer.enhance(factor)

        img_save = img.save(image_path)

        predictions = model(image_path).pandas().xyxy[0]

        classes = []
        for card_pred in predictions.name:
            if card_pred not in classes:
                classes.append(card_pred)
            if len(classes) == 9:
                break
        print(classes)
        if len(classes) < 9:
            return khang_img

        hands = []
        for cl in classes:
            card = cl[:-1]
            if cl[-1] == "S":
                card = "spade_" + card
            elif cl[-1] == "C":
                card = "club_" + card
            elif cl[-1] == "D":
                card = "diamond_" + card
            elif cl[-1] == "H":
                card = "heart_" + card
            hands.append(card)

        sort_n2(hands)


        kind = four_of_a_kind(hands)
        if kind:
            fig = plt.figure(figsize=(12, 3))
            four = [_ + "_" + kind for _ in suits]
            for i, card in enumerate(four):
                card_path = path_52cards + prefix + card + suffix
                img = mpimg.imread(card_path)
                fig.add_subplot(1, 4, i+1)
                plt.axis("off")
                plt.imshow(img)
            fig.savefig(result_path)
            im = Image.open(result_path)
            plt.axis("off")
            plt.show()
            return im

        max_hands = vip_permutations(hands)
        im = show_hand(max_hands, result_path)
    except Exception as e:
        im = mpimg.imread(khang_path)
        print(e)
    return im

In [7]:
# demo = gr.Interface(
#     flip, 
#     inputs=[gr.inputs.Image(type="pil", label="Input Image", source="webcam"),], 
#     outputs="image",
#     live=True
# )

demo = gr.Interface(
    flip, 
    inputs=[gr.inputs.Image(type="pil", label="Input Image"),], 
    outputs="image",
)

demo.launch(share=True)

Running on local URL:  http://127.0.0.1:7861
Running on public URL: https://db737a13-98a0-4d7d.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades (NEW!), check out Spaces: https://huggingface.co/spaces


['10C', 'KS', 'KH', '9D', 'QS', 'KD', '6S', 'AS', '7H']
['QC', '10C', '7H', '6S', 'QS', '4C', 'AS', '9S', '3H']
['5H', '10H', 'AH', 'JH', '10S', '3C', '9H', '4S', 'KD']


In [7]:
import shutil
import os

if os.path.exists(test_path):
    shutil.rmtree(test_path)
if os.path.exists(results_path):
    shutil.rmtree(results_path)

os.mkdir(test_path)
os.mkdir(results_path)

['3C', 'JH', '10S', '10H', 'AH', '9H', '5H', 'KD', '4S']
['5C', 'QS', '9D', 'AH', '10D', '2D', '7C', '4S', 'KS']
